<h1><b>This Jupiter Notebook will be used to contain the Neighbourhood Selection project for the Coursera Data Science Capstone Project</b></h1>

In [2]:
import pandas as pd #Import Pandas with pd as alias.

In [3]:
import numpy as np #Import Numpy with np as alias.

In [4]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


<h3><i>Step 1: Webscrape Toronto location data from Postal list site</i></h3>

In [5]:
# Get additional libraries
!pip install bs4
!pip install requests


/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1272 sha256=e07a098979f6cd7399adef17cbd20891a10877cdbbee04effa1b4ee5d684a5c6
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/0a/9e/ba/20e5bbc1afef3a491f0b3bb74d508f99403aabe76eda2167ca
Successfully built bs4
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.

In [6]:
from bs4 import BeautifulSoup # this module helps in web scrapping.
import requests  # this module helps us to download a web page

In [79]:
# Screen Grab. Add raw data column to help descramble borough/Neighborhood issues M3B,M3C,M3K,M3L.M3M,M3N,
Torontodf = pd.DataFrame(columns=['PostalCode','Borough','Neighborhood','Raw'])
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
# get the contents of the webpage in text format and store in a variable called data
data  = requests.get(url).text
soup = BeautifulSoup(data,"html5lib")
#find a html table in the web page
table = soup.find('table') # in html table is represented by the tag <table>
for row in table.findAll('td'):
    cell = {}    
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Raw'] = repr(row.span)
        cell['Borough'] = (row.span.text).split('(')[0]
#        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        cell['Neighborhood'] = row.span.text.split('(')[1].replace(')',' ').replace(' /',',').strip(')')
        Torontodf = Torontodf.append({"PostalCode":cell['PostalCode'], "Borough":cell['Borough'], "Neighborhood":cell['Neighborhood'], "Raw":cell['Raw']}, ignore_index=True)
        if row.p.text[:3] == 'M3C':
            print(row.span)


<span style="font-size:85%;"><a href="/wiki/North_York" title="North York">North York</a><br/>(<a href="/wiki/Don_Mills" title="Don Mills">Don Mills</a>)<br/>South<br/>(<a href="/wiki/Flemingdon_Park" title="Flemingdon Park">Flemingdon Park</a>)</span>


Clean Webscraped data.

In [100]:
# List of Borough replacements
Torontodf['Borough']=Torontodf['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
Torontodf.loc[Torontodf.PostalCode == 'M3C', 'Neighborhood'] = "Don Mills South, Flemingdon Park" # 2nd hood dropped in scrape
Torontodf.loc[Torontodf.PostalCode == 'M4J', 'Borough'] = "East York, East Toronto" # Postal code has 2 Boroughs can't drop / character.
Torontodf.loc[Torontodf.PostalCode == 'M3K', 'Neighborhood'] = "Downsview East, CFB Toronto" # 2nd hood dropped in scrape
Torontodf.loc[Torontodf.PostalCode == 'M5P', 'Neighborhood'] = "Forest Hill North, Forest Hill West" # Guess that Forest Hill North & West is FHN and FHW..
Torontodf.loc[Torontodf.PostalCode == 'M7R', 'Neighborhood'] = "Canada Post Gateway Processing Centre" # Cleaned up name
Torontodf.loc[Torontodf.PostalCode == 'M5W', 'Neighborhood'] = "Canada Post PO Boxes" # Cleaned up name
Torontodf.loc[Torontodf.PostalCode == 'M7Y', 'Neighborhood'] = "Canada Post Business Processing Centre" # Cleaned up name
Torontodf.loc[Torontodf.PostalCode == 'M7Y', 'Borough'] = "East Toronto" # Cleaned up name

#Drop Canada Post records because they aren't proper neighborhoods.
Torontodf.drop(Torontodf[Torontodf['PostalCode'] == 'M7R'].index, inplace = True)
Torontodf.drop(Torontodf[Torontodf['PostalCode'] == 'M5W'].index, inplace = True)
Torontodf.drop(Torontodf[Torontodf['PostalCode'] == 'M7Y'].index, inplace = True)

#Torontodf.loc[Torontodf['PostalCode'].str.endswith('Z')]

#print(Torontodf['Neighborhood'])
#print(Torontodf.loc[Torontodf['PostalCode'].isin(['M3B','M3C','M3K','M4J','M3L','M3M','M3N'])])
#Torontodf.loc[Torontodf['PostalCode'].isin(['M3B','M3C','M3K','M4J','M3L','M3M','M3N'])]

In [101]:
Torontodf.shape

(100, 4)